In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
df = pd.read_csv('./Incidents_Responded_to_by_Fire_Companies.csv')

C:\Users\Yuan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
df.tail()

,IM_INCIDENT_KEY,FIRE_BOX,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,UNITS_ONSCENE,LAST_UNIT_CLEARED_DATE_TIME,HIGHEST_LEVEL_DESC,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,...,CO_DETECTOR_PRESENT_DESC,FIRE_ORIGIN_BELOW_GRADE_FLAG,STORY_FIRE_ORIGIN_COUNT,FIRE_SPREAD_DESC,DETECTOR_PRESENCE_DESC,AES_PRESENCE_DESC,STANDPIPE_SYS_PRESENT_FLAG,distribution_times,Time,Hour
2277774,62589520,2225,"300 - Rescue, EMS incident, other",12/27/2017 12:06:29 PM,NaN,NaN,12/27/2017 12:23:57 PM,"1 - More than initial alarm, less than Signal 7-5",1048.0,"00 - Action taken, other",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2017-12-27 12:06:29,12:06:29
2277775,62589522,3757,735B - Unnecessary alarm/alarm system testing ...,12/27/2017 12:08:21 PM,NaN,NaN,12/27/2017 12:25:33 PM,"1 - More than initial alarm, less than Signal 7-5",1032.0,86 - Investigate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2017-12-27 12:08:21,12:08:21
2277776,62589542,715,745A - Unnecessary alarm/ordinary household ac...,12/27/2017 12:05:10 PM,NaN,NaN,12/27/2017 12:32:09 PM,"1 - More than initial alarm, less than Signal 7-5",1619.0,86 - Investigate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2017-12-27 12:05:10,12:05:10
2277777,62589549,936,735A - Unwarranted alarm/defective condition o...,12/27/2017 12:13:40 PM,NaN,NaN,12/27/2017 12:25:36 PM,"1 - More than initial alarm, less than Signal 7-5",716.0,86 - Investigate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2017-12-27 12:13:40,12:13:40
2277778,62589556,1031,735A - Unwarranted alarm/defective condition o...,12/27/2017 12:19:51 PM,NaN,NaN,12/27/2017 12:39:15 PM,"1 - More than initial alarm, less than Signal 7-5",1164.0,86 - Investigate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2017-12-27 12:19:51,12:19:51


In [26]:
df.count()

IM_INCIDENT_KEY                 2277779
FIRE_BOX                        2277770
INCIDENT_TYPE_DESC              2277779
INCIDENT_DATE_TIME              2277779
ARRIVAL_DATE_TIME               2222561
UNITS_ONSCENE                   2222367
LAST_UNIT_CLEARED_DATE_TIME     2277777
HIGHEST_LEVEL_DESC              2277640
TOTAL_INCIDENT_DURATION         2277643
ACTION_TAKEN1_DESC              2277779
ACTION_TAKEN2_DESC               480698
ACTION_TAKEN3_DESC               222052
PROPERTY_USE_DESC               2277773
STREET_HIGHWAY                  2239622
ZIP_CODE                        2277771
BOROUGH_DESC                    2277779
FLOOR                            487407
CO_DETECTOR_PRESENT_DESC          29675
FIRE_ORIGIN_BELOW_GRADE_FLAG       1909
STORY_FIRE_ORIGIN_COUNT           13119
FIRE_SPREAD_DESC                  13118
DETECTOR_PRESENCE_DESC            13117
AES_PRESENCE_DESC                 13117
STANDPIPE_SYS_PRESENT_FLAG        13121
dtype: int64

In [16]:
#What proportion of FDNY responses in this dataset correspond to the most common type of incident?
m = df.INCIDENT_TYPE_DESC.value_counts().max()/df.shape[0]
print(m)

0.3614828304238471


In [33]:
#What is the ratio of the average number of units that arrive to a scene of an incident classified as '111 - Building fire' 
#to the number that arrive for '651 - Smoke scare, odor of smoke'?
ratio = df[df['INCIDENT_TYPE_DESC'] == '111 - Building fire']['UNITS_ONSCENE'].mean() / df[df['INCIDENT_TYPE_DESC'] == '651 - Smoke scare, odor of smoke']['UNITS_ONSCENE'].mean()
print(ratio)

2.7597595139775324


In [41]:
#the ratio of Staten Island false call rate to Manhattan false call rate.
df[(df['INCIDENT_TYPE_DESC'] == '710 - Malicious, mischievous false call, other')]['BOROUGH_DESC'].value_counts()

4 - Brooklyn         32110
2 - Bronx            26467
5 - Queens           25771
1 - Manhattan        25240
3 - Staten Island     8276
Name: BOROUGH_DESC, dtype: int64

In [348]:
ratio = 8276/25240
print(round(ratio,10))

0.3278922345


In [48]:
import time

In [64]:
df['distribution_times'] = pd.to_datetime(df['ARRIVAL_DATE_TIME']) - pd.to_datetime(df['INCIDENT_DATE_TIME'])

In [89]:
df['Time'] = pd.to_datetime(df['INCIDENT_DATE_TIME'])

In [106]:
from datetime import datetime
df['Hour'] = df['Time'].dt.time

In [123]:
def hr_func(ts):
    return ts.hour

In [125]:
df['Hour'] = df['Time'].apply(hr_func)

In [77]:
df[['distribution_times']].describe()

,distribution_times
count,2222561
mean,0 days 00:05:05.505618
std,0 days 00:12:09.148359
min,-1 days +22:15:18
25%,0 days 00:03:38
50%,0 days 00:04:34
75%,0 days 00:05:44
max,6 days 22:44:58


In [127]:
5+44/60

5.733333333333333

In [161]:
# Find the hour of the day that has the highest proportion of cooking fires and submit that proportion of cooking fires. 
df1 = df[df['INCIDENT_TYPE_DESC'] == '113 - Cooking fire, confined to container']

18    6806
17    6541
19    6456
16    5680
20    5451
15    5150
12    5000
14    4979
13    4963
11    4464
21    4284
10    3983
9     3503
22    3140
8     2818
23    2530
0     2047
7     2007
1     1571
6     1455
2     1203
3     1059
5      998
4      951
Name: Hour, dtype: int64

In [166]:
ds = df1.groupby('Hour').count()
ds['INCIDENT_TYPE_DESC']

Hour
0     2047
1     1571
2     1203
3     1059
4      951
5      998
6     1455
7     2007
8     2818
9     3503
10    3983
11    4464
12    5000
13    4963
14    4979
15    5150
16    5680
17    6541
18    6806
19    6456
20    5451
21    4284
22    3140
23    2530
Name: INCIDENT_TYPE_DESC, dtype: int64

In [167]:
DS = df.groupby('Hour').count()
rate = ds['INCIDENT_TYPE_DESC']/DS['INCIDENT_TYPE_DESC']
print(rate)

Hour
0     0.028577
1     0.027372
2     0.025553
3     0.025634
4     0.023980
5     0.023848
6     0.027909
7     0.027618
8     0.030831
9     0.034045
10    0.036387
11    0.039182
12    0.042661
13    0.040711
14    0.039511
15    0.040740
16    0.044270
17    0.047996
18    0.050847
19    0.050522
20    0.045150
21    0.038825
22    0.031701
23    0.028596
Name: INCIDENT_TYPE_DESC, dtype: float64


In [142]:
rate = round(6806/133853,10)
print(rate)

0.0508468245


In [193]:
#the coefficient of determination (R squared) between the number of residents at each zip code and the number of inicidents 
#whose type is classified as '111 - Building fire' at each of those zip codes. 
#ds1 = df[df['INCIDENT_TYPE_DESC'] == '111 - Building fire'].groupby('ZIP_CODE').count()

In [199]:
ds1 = df[df['INCIDENT_TYPE_DESC'] == '111 - Building fire']

In [244]:
ds11= ds1[['ZIP_CODE','INCIDENT_TYPE_DESC']]

In [245]:
ds11['ZIP_CODE'] = ds11['ZIP_CODE'].astype(int)

C:\Users\Yuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [178]:
cencus = pd.read_csv('./2010+Census+Population+By+Zipcode+(ZCTA).csv')

In [285]:
cencus[cencus['ZIP_CODE'] == 10001]

,ZIP_CODE,2010 Census Population
2429,10001,21102


In [214]:
cencus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33092 entries, 0 to 33091
Data columns (total 2 columns):
ZIP_CODE                  33092 non-null int64
2010 Census Population    33092 non-null int64
dtypes: int64(2)
memory usage: 517.1 KB


In [246]:
ds11.columns, cencus.columns

(Index(['ZIP_CODE', 'INCIDENT_TYPE_DESC'], dtype='object'),
 Index(['ZIP_CODE', '2010 Census Population'], dtype='object'))

In [295]:
temp = ds11.groupby('ZIP_CODE').count()


C:\Users\Yuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [312]:
res = pd.DataFrame({'ZIP_CODE': sorted(ds11['ZIP_CODE'].unique()),'COUNT': temp['INCIDENT_TYPE_DESC'].values})
res.head()

,ZIP_CODE,COUNT
0,10001,83
1,10002,114
2,10003,82
3,10004,8
4,10005,5


In [313]:
data = pd.merge(res,cencus, how='left',on='ZIP_CODE')

In [317]:
data.columns

Index(['ZIP_CODE', 'COUNT', '2010 Census Population'], dtype='object')

In [319]:
data.dropna(inplace=True)

In [325]:
data.shape

(173, 3)

In [263]:
from sklearn.metrics import r2_score

In [352]:
round(-r2_score(data['2010 Census Population'],data['COUNT']),10)

3.2150888973

In [328]:
#CO Detector present or not
df['DETECTOR_PRESENCE_DESC'].value_counts()

N - None present    9358
1 - Present         3759
Name: DETECTOR_PRESENCE_DESC, dtype: int64

In [336]:
df_CO = df[['DETECTOR_PRESENCE_DESC','TOTAL_INCIDENT_DURATION']]

In [346]:
df_CO.dropna(inplace=True)

C:\Users\Yuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [347]:
df_CO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12998 entries, 90 to 2276808
Data columns (total 2 columns):
DETECTOR_PRESENCE_DESC     12998 non-null object
TOTAL_INCIDENT_DURATION    12998 non-null float64
dtypes: float64(1), object(1)
memory usage: 304.6+ KB


In [376]:
df_CO.shape

(12998, 3)

In [354]:
df_CO['Duration'] = df['TOTAL_INCIDENT_DURATION']/60

C:\Users\Yuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [359]:
#Calculate the chi-square test statistic for testing whether an incident is more likely to last longer than 60 minutes when CO detector is not present. 
#Again only consider incidents that have information about whether a CO detector was present or not.

In [363]:
s1 = df_CO[df_CO['DETECTOR_PRESENCE_DESC'] == 'N - None present']['Duration']
s2 = df_CO[df_CO['DETECTOR_PRESENCE_DESC'] == '1 - Present']['Duration']

In [372]:
from scipy.stats import chi2

In [396]:
chi_squared_stat = (((s1.values-60)**2)/60).sum()

In [397]:
print(chi_squared_stat)

10346742.806907408


In [387]:
crit = chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 1)   # Df = number of variable categories - 1

In [388]:
print("Critical value")
print(crit)

p_value = 1 - chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=1)
print("P value")
print(p_value)

Critical value
3.841458820694124
P value
0.0


## 